In [1]:
#@title 安裝相關套件 (yt-dlp, openAI API, Pydub)
! pip install --upgrade pip
! pip install yt-dlp
! pip install openai
! pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#@title 下載 youtube 影片 
import yt_dlp

#@markdown ### 輸出檔案名稱 (如輸入 audio 會產生 audio.mp3):
filename = 'audio' #@param {type:"string"} 

# 填入影片的URL
#@markdown ### Youtube 連結:
url = 'https://www.youtube.com/watch?v=PRp3kJtMQwU' #@param {type:"string"} 


# 設定選項
ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': filename , 
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
}

# 建立 yt_dlp 下載器物件
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

[youtube] Extracting URL: https://www.youtube.com/watch?v=PRp3kJtMQwU
[youtube] PRp3kJtMQwU: Downloading webpage
[youtube] PRp3kJtMQwU: Downloading android player API JSON
[info] PRp3kJtMQwU: Downloading 1 format(s): 251
[dashsegments] Total fragments: 3
[download] Destination: audio
[download] 100% of   22.96MiB in 00:00:00 at 30.31MiB/s              
[ExtractAudio] Destination: audio.mp3
Deleting original file audio (pass -k to keep)


In [3]:
#@title 分割 youtube 影片 
from pydub import AudioSegment

# 設定每個分割檔案的長度（單位：毫秒）
#@markdown ### 分割檔案的長度（單位：毫秒）:
segment_length = 1000000 #@param {type:"integer"}


# 載入 MP3 音檔
sound = AudioSegment.from_file(f'{filename}.mp3', format='mp3')

sound_track = []
# 將音檔分割成多個檔案
for i, chunk in enumerate(sound[::segment_length]):
    # 設定分割檔案的檔名
    chunk.export(f'output_{i}.mp3', format='mp3')
    audio_file = open(f'output_{i}.mp3', "rb")
    sound_track.append(audio_file)
    


In [4]:
#@title 語音轉文字
import openai
#@markdown ### 填入 OpenAI API Secret Key:
openai.api_key = '' #@param {type:"string"}

transcripts = []
for audio_file in sound_track:
  transcript = openai.Audio.transcribe("whisper-1", audio_file)
  transcripts.append(transcript.to_dict().get('text') )


transcripts_out = ' '.join(transcripts)

transcript_chunks = []

ret = ''
for script in transcripts_out.split():
    ret = ret + ' ' + script
    if len(ret) > 1000:
        transcript_chunks.append(ret)
        ret = ''
transcript_chunks.append(ret)

print(f'分成 {len(transcript_chunks)} 段轉錄結果')

分成 9 段轉錄結果


In [5]:
#@title 用ChatGPT 摘要文章


#@markdown ### 請輸入咒語:
prompt = "請你成為文章摘要的小幫手，摘要以下文字，以繁體中文輸出" #@param {type:"string"}

result_ary = []

for t in transcript_chunks:
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": t}
      ]
    )
    result_ary.append(completion.choices[0].message)
    print(completion.choices[0].message.get('content'))
    print()

本文是一集介紹在日本買房地產的節目，主持人邀請了專業的資深老師淡如姐分享他的心得。淡如姐表示，他在2010年開始進軍日本房地產，主要是因為金融風險太大，讓他意識到金融投資的風險，所以轉向投資實體房地產。另外，淡如姐也提到，對於想要在日本買房的人來說，需要有一定的資金基礎和專業知識，才能夠避免風險和獲取投資成功的收益。

這段文字講述在日本長居的經驗，作者提到了開設公司的費用以及需要繳納高額稅金的問題。作者也谈到在日本房地产投资方面的困难，需要从台湾的信托公司贷款，并且房子价格非常高。作者最后提到自己在购买物业时专注于购买居酒屋，并且参考了Google评分以及租金投报率等因素来进行决策。

這篇文章介紹了東京房地產的投資機會，特別是位於文京區附近的一間住宅。作者建議投資人先選擇穩定的地段，離市中心不要太遠，且盡量避免位於液化區域。此外，投資人應該詳細瞭解租戶的情況，尤其是他們的租賃歷史和背景資料。文章最後提到，套房有時表面的投報率可能不是真實的，投資人應該尋找穩定的上班族作為租戶。

文章提到日本房產投資的一些問題和建議。首先，小公寓的維修費用和管理費比較高，容易有租客逃租和留下遺物等問題，建議購買2LDK或以上的房型。其次，建議找專業建築師評估房產，預測貸款期間的房租收益和未來漲幅，投資長期看好，可以考慮購買經營穩定的商業場所，如居酒屋。最後，老房子年代越久，稅金越少，有投資優勢。

本文提到作者曾買過一間陽台有10坪的房子，但要扣除管理費用和修繕費用等成本，投報率其實只有5%以上。日本房市漲幅比台灣慢，而且很多年輕人因大地震等因素不想買房，導致日本房價長期走低。買日本房地產最好的方法是出租，但要注意成本和稅費。作者也提到，最好不要買套房，而帶租約的營業場合比較穩定，投報率也相對較高。

本文探討在日本購買房產時需要注意的管理費、修繕激勵金等費用。作者提到，在選擇房產時，需要實際去看一下，以免買到風化區等不良區域，還要注意房屋是否需要拉皮等大裝修，以及修繕激勵金和管理費的多寡，以避免投資失敗。其中，修繕激勵金需要每個月繳交，而管理費則包括有存錢和花費等兩種情況，需仔細查看管理委員會是否盡職，以及管理員是否收取過高費用。

此文中，作者分享了自己的投資經驗。他提到在投資日本房地產時，要注意地點和房型，因為租賃率和投報率有重要關係。他也談到買房子的費用，如仲介費和換租客